In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/workspace/deep_birth_death/src')

In [3]:
import pickle
import numpy as np
import pandas as pd
import random
from time import time

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, GlobalAveragePooling1D, MaxPooling1D, Concatenate
from tensorflow.keras.layers import Dense, Conv1D, Dropout, BatchNormalization, LeakyReLU, ELU
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [4]:
def shuffle_three_arrays(a, b, c):
    combined = list(zip(a, b, c))
    random.shuffle(combined)
    a_permuted, b_permuted, c_permuted = zip(*combined)
    
    return np.array(a_permuted), np.array(b_permuted), np.array(c_permuted)

# Load data

In [5]:
n_tips = ['674', '489', '87']
pickle_base = '/workspace/deep_birth_death/deep_learning/pickles/old_sims/dataset_'
data = dict()
for i in n_tips:
    with open(pickle_base + i + "_10k.pkl", 'rb') as f:
        data[i] = pickle.load(f)

In [6]:
max_size = 674


x_train_total= {k: [] for k in ["BD", "HE", "ME", "SAT", "SR", "WW"]}
x_test_total= {k: [] for k in ["BD", "HE", "ME", "SAT", "SR", "WW"]}
y_train_total= {k: [] for k in ["BD", "HE", "ME", "SAT", "SR", "WW"]}
y_test_total= {k: [] for k in ["BD", "HE", "ME", "SAT", "SR", "WW"]}
resc_fact_train_total = {k: [] for k in ["BD", "HE", "ME", "SAT", "SR", "WW"]}
resc_fact_test_total = {k: [] for k in ["BD", "HE", "ME", "SAT", "SR", "WW"]}

for tip in data:

    data[tip]['X_train'] = np.pad(data[tip]['X_train'], 
                                 pad_width=((0,0), (0, max_size - data[tip]['X_train'].shape[1])),
                                 mode='constant', constant_values=0)
    data[tip]['X_test'] = np.pad(data[tip]['X_test'], 
                                pad_width=((0,0), (0, max_size - data[tip]['X_test'].shape[1])),
                                mode='constant', constant_values=0)

    for label in np.unique(data[tip]['div_info_train']):
        div_scenario = label.split('/')[1].split('_')[0]

        x_train = data[tip]['X_train'][data[tip]['div_info_train'] == label]
        x_test = data[tip]['X_test'][data[tip]['div_info_test'] == label]

        y_train = data[tip]['y_reg_norm_train'][data[tip]['div_info_train'] == label]
        y_test = data[tip]['y_reg_test'][data[tip]['div_info_test'] == label]
        
        resc_fact_train = data[tip]['resc_factor_train'][data[tip]['div_info_train'] == label]
        resc_fact_test = data[tip]['resc_factor_test'][data[tip]['div_info_test'] == label]

        if div_scenario in x_train_total:
            x_train_total[div_scenario].extend(x_train)
            x_test_total[div_scenario].extend(x_test)
            y_train_total[div_scenario].extend(y_train)
            y_test_total[div_scenario].extend(y_test)
            resc_fact_train_total[div_scenario].extend(resc_fact_train)
            resc_fact_test_total[div_scenario].extend(resc_fact_test)

In [7]:
for k in x_train_total:
    x_train_total[k] = np.array(x_train_total[k])
    x_test_total[k] = np.array(x_test_total[k])
    y_train_total[k] = np.array(y_train_total[k])
    y_test_total[k] = np.array(y_test_total[k])
    resc_fact_train_total[k] = np.array(resc_fact_train_total[k])
    resc_fact_test_total[k] = np.array(resc_fact_test_total[k])

    x_train_total[k], y_train_total[k], resc_fact_train_total[k] = shuffle_three_arrays(x_train_total[k], y_train_total[k], resc_fact_train_total[k])
    x_test_total[k], y_test_total[k], resc_fact_test_total[k] = shuffle_three_arrays(x_test_total[k], y_test_total[k], resc_fact_test_total[k])
                                                          
    print(k)
    print(x_train_total[k].shape)
    print(y_train_total[k].shape)  
    print(resc_fact_train_total[k].shape)
    print(x_test_total[k].shape )
    print(y_test_total[k].shape )
    print(resc_fact_test_total[k].shape)

BD
(26904, 674)
(26904, 2)
(26904,)
(3096, 674)
(3096, 2)
(3096,)
HE
(26994, 674)
(26994, 2)
(26994,)
(3006, 674)
(3006, 2)
(3006,)
ME
(27056, 674)
(27056, 4)
(27056,)
(2944, 674)
(2944, 4)
(2944,)
SAT
(27036, 674)
(27036, 1)
(27036,)
(2964, 674)
(2964, 1)
(2964,)
SR
(26972, 674)
(26972, 5)
(26972,)
(3028, 674)
(3028, 5)
(3028,)
WW
(27038, 674)
(27038, 5)
(27038,)
(2962, 674)
(2962, 5)
(2962,)


# Train the regression model

In [9]:
def create_nn(n_out, div_scenario = None):
    input_data = Input(shape=(674, 1))


    x = Conv1D(16, kernel_size=3, padding='same')(input_data)
    x = ELU()(x)
    x = Conv1D(16, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Conv1D(32, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = Conv1D(32, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Conv1D(64, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = Conv1D(64, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    x = Conv1D(128, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = Dropout(0.3)(x)

    x = GlobalAveragePooling1D()(x)

    x = Dense(32)(x)
    x = ELU()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(n_out)(x)


    if div_scenario != "SAT":
        out_list = []
        for i in range(n_out):
            y = Dense(32)(x)
            y = ELU()(y)
            y = Dropout(0.3)(y)
            y = Dense(1)(y)
            y = LeakyReLU(alpha=10)(y)
            out_list.append(y)
        output_class = Concatenate()(out_list)
        
    elif div_scenario == "WW":
        out_list = []
        for i in range(n_out):
            y = Dense(32)(x)
            y = ELU()(y)
            y = Dropout(0.3)(y)
            y = Dense(1)(y)
            y = Linear(y)
            out_list.append(y)
        output_class = Concatenate()(out_list)                
        
    else:
        x = Dense(32)(x)
        x = ELU()(x)
        x = Dropout(0.3)(x)
        x = Dense(1)(x)
        output_class = LeakyReLU(alpha=10)(x)
            
    return Model(input_data, output_class)

In [ ]:
out_folder_path = "/workspace/deep_birth_death/deep_learning/models/testing_total_reg/"
callback = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

for div_scenario in x_train_total:
    
    print("\nRegression training: ", div_scenario)
    
    nn_model = create_nn(len(y_train_total[div_scenario][0]), div_scenario=div_scenario)
    nn_model.compile(loss="mae", optimizer=Adam(learning_rate=0.001), metrics=['mse'])    
    
    start = time()                     
    history = nn_model.fit(x_train_total[div_scenario],
                           y_train_total[div_scenario],
                           batch_size=128, epochs=1000, validation_split=0.1,
                           callbacks=[callback], verbose=1)                     
    elapsed_time = time()-start  
                         
    print('Elapsed time', elapsed_time)
              

    #Save the results
    save_path = out_folder_path + div_scenario + "_TOTAL_regression_" 
                         
    nn_model.save(save_path + "model.keras") 
    with open(save_path + "history.pkl", 'wb') as f:
        pickle.dump(history.history, f)
    with open(save_path + "model_data.pkl", 'wb') as f:
            pickle.dump([nn_model.count_params(), elapsed_time], f)
                    


Regression training:  BD
Epoch 1/1000
190/190 [==============================] - 9s 42ms/step - loss: 0.1409 - mse: 0.1399 - val_loss: 0.0776 - val_mse: 0.0102
Epoch 2/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0850 - mse: 0.0115 - val_loss: 0.0719 - val_mse: 0.0089
Epoch 3/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0799 - mse: 0.0104 - val_loss: 0.0672 - val_mse: 0.0079
Epoch 4/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0772 - mse: 0.0099 - val_loss: 0.0667 - val_mse: 0.0079
Epoch 5/1000
190/190 [==============================] - 8s 42ms/step - loss: 0.0752 - mse: 0.0096 - val_loss: 0.0703 - val_mse: 0.0086
Epoch 6/1000
190/190 [==============================] - 8s 40ms/step - loss: 0.0740 - mse: 0.0094 - val_loss: 0.0687 - val_mse: 0.0082
Epoch 7/1000
190/190 [==============================] - 8s 40ms/step - loss: 0.0724 - mse: 0.0091 - val_loss: 0.0697 - val_mse: 0.0087
Epoch 8/1000
190/190 [=======

190/190 [==============================] - 8s 42ms/step - loss: 0.0601 - mse: 0.0067 - val_loss: 0.0608 - val_mse: 0.0070
Epoch 62/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0599 - mse: 0.0067 - val_loss: 0.0625 - val_mse: 0.0072
Epoch 63/1000
190/190 [==============================] - 8s 42ms/step - loss: 0.0594 - mse: 0.0066 - val_loss: 0.0608 - val_mse: 0.0070
Epoch 64/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0596 - mse: 0.0066 - val_loss: 0.0605 - val_mse: 0.0068
Epoch 65/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0589 - mse: 0.0065 - val_loss: 0.0612 - val_mse: 0.0070
Epoch 66/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0591 - mse: 0.0065 - val_loss: 0.0618 - val_mse: 0.0072
Epoch 67/1000
190/190 [==============================] - 8s 40ms/step - loss: 0.0589 - mse: 0.0065 - val_loss: 0.0647 - val_mse: 0.0078
Epoch 68/1000
190/190 [==============================] - 8s 41

190/190 [==============================] - 8s 41ms/step - loss: 0.0498 - mse: 0.0050 - val_loss: 0.0693 - val_mse: 0.0094
Epoch 122/1000
190/190 [==============================] - 8s 40ms/step - loss: 0.0490 - mse: 0.0048 - val_loss: 0.0661 - val_mse: 0.0082
Epoch 123/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0489 - mse: 0.0048 - val_loss: 0.0664 - val_mse: 0.0081
Epoch 124/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0487 - mse: 0.0048 - val_loss: 0.0659 - val_mse: 0.0083
Epoch 125/1000
190/190 [==============================] - 8s 40ms/step - loss: 0.0485 - mse: 0.0047 - val_loss: 0.0661 - val_mse: 0.0084
Epoch 126/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0480 - mse: 0.0047 - val_loss: 0.0673 - val_mse: 0.0086
Epoch 127/1000
190/190 [==============================] - 8s 42ms/step - loss: 0.0492 - mse: 0.0048 - val_loss: 0.0651 - val_mse: 0.0080
Epoch 128/1000
190/190 [==============================] 

Epoch 25/1000
190/190 [==============================] - 8s 40ms/step - loss: 0.0221 - mse: 7.4973e-04 - val_loss: 0.0219 - val_mse: 7.2979e-04
Epoch 26/1000
190/190 [==============================] - 7s 39ms/step - loss: 0.0220 - mse: 7.4426e-04 - val_loss: 0.0215 - val_mse: 7.1981e-04
Epoch 27/1000
190/190 [==============================] - 8s 40ms/step - loss: 0.0221 - mse: 7.5088e-04 - val_loss: 0.0217 - val_mse: 7.3373e-04
Epoch 28/1000
190/190 [==============================] - 7s 39ms/step - loss: 0.0220 - mse: 7.4496e-04 - val_loss: 0.0217 - val_mse: 7.2126e-04
Epoch 29/1000
190/190 [==============================] - 8s 40ms/step - loss: 0.0219 - mse: 7.3947e-04 - val_loss: 0.0214 - val_mse: 7.0923e-04
Epoch 30/1000
190/190 [==============================] - 8s 42ms/step - loss: 0.0217 - mse: 7.3147e-04 - val_loss: 0.0214 - val_mse: 7.1061e-04
Epoch 31/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0218 - mse: 7.3686e-04 - val_loss: 0.0219 - val_mse: 7.3

Epoch 82/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0201 - mse: 6.6104e-04 - val_loss: 0.0222 - val_mse: 7.6763e-04
Epoch 83/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0201 - mse: 6.6223e-04 - val_loss: 0.0220 - val_mse: 7.5837e-04
Epoch 84/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0202 - mse: 6.6476e-04 - val_loss: 0.0216 - val_mse: 7.2939e-04
Epoch 85/1000
190/190 [==============================] - 8s 42ms/step - loss: 0.0201 - mse: 6.5976e-04 - val_loss: 0.0217 - val_mse: 7.3762e-04
Epoch 86/1000
190/190 [==============================] - 8s 40ms/step - loss: 0.0200 - mse: 6.6043e-04 - val_loss: 0.0222 - val_mse: 7.7415e-04
Epoch 87/1000
190/190 [==============================] - 8s 40ms/step - loss: 0.0201 - mse: 6.6075e-04 - val_loss: 0.0219 - val_mse: 7.7074e-04
Epoch 88/1000
190/190 [==============================] - 8s 42ms/step - loss: 0.0200 - mse: 6.5714e-04 - val_loss: 0.0215 - val_mse: 7.2

190/190 [==============================] - 7s 39ms/step - loss: 0.0166 - mse: 5.0449e-04 - val_loss: 0.0232 - val_mse: 8.2952e-04
Epoch 139/1000
190/190 [==============================] - 8s 40ms/step - loss: 0.0167 - mse: 5.0209e-04 - val_loss: 0.0236 - val_mse: 8.8154e-04
Epoch 140/1000
190/190 [==============================] - 8s 42ms/step - loss: 0.0167 - mse: 5.0136e-04 - val_loss: 0.0243 - val_mse: 9.2057e-04
Epoch 141/1000
190/190 [==============================] - 8s 42ms/step - loss: 0.0165 - mse: 4.9390e-04 - val_loss: 0.0231 - val_mse: 8.4531e-04
Epoch 142/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0164 - mse: 4.9105e-04 - val_loss: 0.0228 - val_mse: 8.1983e-04
Epoch 143/1000
190/190 [==============================] - 8s 40ms/step - loss: 0.0165 - mse: 4.9182e-04 - val_loss: 0.0231 - val_mse: 8.3438e-04
Epoch 144/1000
190/190 [==============================] - 8s 41ms/step - loss: 0.0162 - mse: 4.8002e-04 - val_loss: 0.0232 - val_mse: 8.4360e-04


191/191 [==============================] - 8s 40ms/step - loss: 2.2024 - mse: 40.5607 - val_loss: 2.2148 - val_mse: 33.8918
Epoch 38/1000
191/191 [==============================] - 8s 40ms/step - loss: 2.1634 - mse: 39.5500 - val_loss: 2.0699 - val_mse: 33.9546
Epoch 39/1000
191/191 [==============================] - 8s 40ms/step - loss: 2.1428 - mse: 38.8369 - val_loss: 2.0599 - val_mse: 35.2646
Epoch 40/1000
191/191 [==============================] - 8s 40ms/step - loss: 2.1222 - mse: 38.3144 - val_loss: 2.0639 - val_mse: 35.2346
Epoch 41/1000
191/191 [==============================] - 8s 40ms/step - loss: 2.1047 - mse: 37.9508 - val_loss: 2.0326 - val_mse: 33.9880
Epoch 42/1000
191/191 [==============================] - 8s 41ms/step - loss: 2.0954 - mse: 37.7338 - val_loss: 2.0252 - val_mse: 32.7080
Epoch 43/1000
191/191 [==============================] - 8s 39ms/step - loss: 2.0866 - mse: 37.0248 - val_loss: 1.9859 - val_mse: 33.9061
Epoch 44/1000
191/191 [=========================

191/191 [==============================] - 8s 39ms/step - loss: 1.4898 - mse: 19.8248 - val_loss: 2.0537 - val_mse: 37.2485
Epoch 97/1000
191/191 [==============================] - 8s 40ms/step - loss: 1.5049 - mse: 20.1437 - val_loss: 2.3455 - val_mse: 42.6758
Epoch 98/1000
191/191 [==============================] - 8s 41ms/step - loss: 1.4829 - mse: 19.4350 - val_loss: 2.0626 - val_mse: 36.1734
Epoch 99/1000
191/191 [==============================] - 8s 41ms/step - loss: 1.4843 - mse: 19.6290 - val_loss: 2.0548 - val_mse: 36.5080
Epoch 100/1000
191/191 [==============================] - 8s 40ms/step - loss: 1.4630 - mse: 19.1026 - val_loss: 2.0590 - val_mse: 39.2017
Epoch 101/1000
191/191 [==============================] - 8s 40ms/step - loss: 1.4475 - mse: 18.8189 - val_loss: 2.1290 - val_mse: 38.3176
Epoch 102/1000
191/191 [==============================] - 8s 40ms/step - loss: 1.4382 - mse: 18.4915 - val_loss: 2.0930 - val_mse: 36.1310
Epoch 103/1000
191/191 [=====================

# Models evaluation

In [ ]:
total_regression_results = dict()

for div_scenario in x_test_total:
    total_regression_results[div_scenario] = dict()
    
    # Load regression model 
    model_path = out_folder_path + div_scenario + "TOTAL_regression_"  
    nn_model = load_model(model_path + 'model.keras')
    
    y_pred = nn_model.predict(x_test_total[div_scenario])

    y_test = y_test_total[div_scenario]     
    
    # Rescale back the values
    if div_scenario == "BD" or div_scenario == "HE" or div_scenario == "SAT":
                    
        y_pred[:, 0] = y_pred[:, 0] * resc_fact_test_total[div_scenario]
    
    elif div_scenario == "ME":

        y_pred[:, 0] = y_pred[:, 0] * resc_fact_test_total[div_scenario]
        y_pred[:, 2] = y_pred[:, 2] * resc_fact_test_total[div_scenario]
  
    else:
                    
        y_pred[:, 0] = y_pred[:, 0] * resc_fact_test_total[div_scenario]
        y_pred[:, 1] = y_pred[:, 1] * resc_fact_test_total[div_scenario]
        y_pred[:, 4] = y_pred[:, 4] * resc_fact_test_total[div_scenario]

    error = abs(y_pred-y_test)
    r_error = abs((y_pred - y_test)/ y_test)
    
    total_regression_results[div_scenario]['MAE'] = np.mean(error, axis=0)
    total_regression_results[div_scenario]['MRE'] = np.mean(r_error, axis=0)

In [ ]:
reg_values = {
    'BD': ['r', 'a'],
    'HE': ['r', 'a'],
    'ME': ['r', 'a', 'time', 'frac'],
    'SR': ['r0', 'r1', 'a0', 'a1', 'time'],
    'WW': ['r0', 'r1', 'a0', 'a1', 'time'],
    'SAT': ['lambda 0'],
}
    
for div_scenario in x_test_total:   
    param_names = reg_values[div_scenario]
    mae_vals = total_regression_results[div_scenario]['MAE']
    mre_vals = total_regression_results[div_scenario]['MRE']
    
    df = pd.DataFrame({
            "param": param_names,
            "MAE": mae_vals,
            "MRE": mre_vals
        })
    
    print("MAE and MRE for :" + div_scenario + " diversification scenario")
    print(df)